# TP3 - Deep Learning Models

In the previous notebooks, we used classic machine learning models based on features computed on the data. Now, we will use Deep Learning to perform sleep staging directly on the raw signals. 

In this tutorial we will use pytorch: https://pytorch.org/

In [1]:
cd "/kaggle/input/automated-sleep-staging-beacon-biosignals-2023-2024"

/kaggle/input/automated-sleep-staging-beacon-biosignals-2023-2024


In [2]:
import pandas as pd
import os
import numpy as np
import random as rd

## Loading the data

First, we load the data from the first EEG channel for each record and the associated hypnogram.

Then, we sample the training and validation records for the current run.

In [3]:
data_for_records = {}
hypnogram_for_records = {}
hypnograms = pd.read_csv('targets_train.csv')
for record in os.listdir("training_records"):
    record_number = int(record[-5])
    x = np.load(f'training_records/{record}')
    data_for_records[record] = x[:,1:250 * 30 + 1]
    hypnogram_for_records[record] = list(hypnograms[hypnograms['record'] == record_number]['target'])
    




In [4]:
rd.seed(1234)
records_list = list(data_for_records)
rd.shuffle(records_list)
training_record,test_records = records_list[:5],records_list[5:]

print('Training records: ',training_record)
print('Test records: ', test_records)

Training records:  ['dreem_4.npy', 'dreem_2.npy', 'dreem_6.npy', 'dreem_5.npy', 'dreem_1.npy']
Test records:  ['dreem_3.npy', 'dreem_0.npy']


In [5]:
def build_dataset(records, data_for_records,hypnogram_for_records):
    X,y = [],[]
    for record in records:
        X.append(data_for_records[record])
        y.extend(hypnogram_for_records[record])

    return np.concatenate(X),y


X_train,y_train = build_dataset(training_record,data_for_records,hypnogram_for_records)
X_test,y_test = build_dataset(test_records,data_for_records,hypnogram_for_records)



## Dataloader and dataset

In this TD, we will only work with one EEG channel.
Let's create dataset functions that will be used for training and testing the model:

*EegEpochDataset*: Eeg Class herited from pytorch Dataset to deal with our data

The dataloader are used by pytorch to randomly sampled elements from the datasets.

In [6]:
""" Load project data
    DataLoader and Dataset for single-channel EEG

"""

import torch
from torch.utils.data import Dataset, DataLoader


def normalize_data(eeg_array):
    """normalize signal between 0 and 1"""

    normalized_array = np.clip(eeg_array, -250, 250)
    normalized_array = normalized_array / 250

    return normalized_array


class EegEpochDataset(Dataset):
    """EEG Epochs dataset."""

    def __init__(self, x_data, y_data, transform=None):
        """
        Args:
            x_data (numpy array): Numpy array of input data.
            y_data (list of numpy array): Sleep Stages
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.y_data = y_data
        self.x_data = x_data
        self.transform = transform

        self.x_data = normalize_data(x_data)

    def __len__(self):
        return len(self.y_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        signal = np.expand_dims(self.x_data[idx], axis=0)
        stage = self.y_data[idx]

        if self.transform:
            signal = self.transform(signal)

        return signal, stage


training_dataset = EegEpochDataset(X_train,y_train)
training_dataloader = DataLoader(training_dataset,batch_size = 32)
validation_dataset = EegEpochDataset(X_test,y_test)
validation_dataloader = DataLoader(validation_dataset,batch_size = 32)


## First CNN model
The model is rather simple, it's a succession of convolution layers with non-linearities.

Then, we apply max-pooling followed by a classification layer.

In [7]:
import torch
import torch.nn as nn


class SingleChannelConvNet(nn.Module):

    def __init__(self):
        super(SingleChannelConvNet, self).__init__()
        self.conv_a = nn.Conv1d(1, 8, 25, stride=5)
        self.conv_b = nn.Conv1d(8, 16, 10, stride=5)
        self.conv_c = nn.Conv1d(16, 32, 10, stride=5)
        self.conv_d = nn.Conv1d(32, 64, 10, stride=5)

        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(64, 5)

    def forward(self, x):

        x = self.relu(self.conv_a(x))
        x = self.relu(self.conv_b(x))
        x = self.relu(self.conv_c(x))
        x = self.relu(self.conv_d(x))
        x = x.max(-1)[0]
        x = self.fc1(x)

        return x


**Question:** 
- What is the downsampling rate of the network ? 
- What does the following line do ?
>x = x.max(-1)[0]
- How many parameters are there in this network ? Is it a lot ?


## Training Loop
We iterate on the training dataloader and train the model.

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

# device: use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# parameters
n_epoch = 30

# neural network and co
my_net = SingleChannelConvNet()
my_net = my_net.to(device) # model into GPU
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(my_net.parameters())
my_net.train()
print('training...')
for epoch in range(n_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(training_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device).float(), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = my_net.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print('epoch %d, %d samples, loss: %.3f' % (epoch + 1, (i+1)*training_dataloader.batch_size,running_loss / (i+1)))
    running_loss = 0.0

print('Finished Training')


training...
epoch 1, 4288 samples, loss: 1.494
epoch 2, 4288 samples, loss: 1.406
epoch 3, 4288 samples, loss: 1.391
epoch 4, 4288 samples, loss: 1.376
epoch 5, 4288 samples, loss: 1.360
epoch 6, 4288 samples, loss: 1.320
epoch 7, 4288 samples, loss: 1.239
epoch 8, 4288 samples, loss: 1.135
epoch 9, 4288 samples, loss: 1.052
epoch 10, 4288 samples, loss: 1.000
epoch 11, 4288 samples, loss: 0.967
epoch 12, 4288 samples, loss: 0.942
epoch 13, 4288 samples, loss: 0.920
epoch 14, 4288 samples, loss: 0.901
epoch 15, 4288 samples, loss: 0.878
epoch 16, 4288 samples, loss: 0.856
epoch 17, 4288 samples, loss: 0.832
epoch 18, 4288 samples, loss: 0.806
epoch 19, 4288 samples, loss: 0.781
epoch 20, 4288 samples, loss: 0.754
epoch 21, 4288 samples, loss: 0.728
epoch 22, 4288 samples, loss: 0.702
epoch 23, 4288 samples, loss: 0.676
epoch 24, 4288 samples, loss: 0.650
epoch 25, 4288 samples, loss: 0.620
epoch 26, 4288 samples, loss: 0.595
epoch 27, 4288 samples, loss: 0.565
epoch 28, 4288 samples, l

## Assessing the model performances

Now the training is complete, let's assess its performance on the validation data

In [9]:
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, confusion_matrix
from sklearn.metrics import confusion_matrix, f1_score
# params
classes = ['Wake', 'N1', 'N2', 'N3', 'REM']

with torch.no_grad():
    prediction_list = torch.empty(0).to(device)
    true_list = torch.empty(0).to(device)
    for data in validation_dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device).float(), labels.to(device)
        
        outputs = my_net(inputs)
        _, predicted = torch.max(outputs, 1)
        prediction_list = torch.cat([prediction_list, predicted])
        true_list = torch.cat([true_list, labels])

true_list = true_list.cpu().numpy()
prediction_list = prediction_list.cpu().numpy()
scores = {'balanced_accuracy': balanced_accuracy_score(true_list, prediction_list),
            'cohen_kappa_score': cohen_kappa_score(true_list, prediction_list),
            'confusion_matrix': confusion_matrix(true_list, prediction_list)}

for elt in scores:
    print(elt)
    print(scores[elt])

balanced_accuracy
0.47550950430996863
cohen_kappa_score
0.4999046450340985
confusion_matrix
[[ 17   1  29  20  43]
 [  2   4  30   4  27]
 [  3  10 481  36 184]
 [  5   1  61 336   5]
 [ 22   1  98  76 393]]


**Question:**
- How does the CNN ranks compared to model from the previous tutorial ? How could you improve it ?
- Where do the mistakes happen ? Plot a few epoch to understand when the network is good or bad.

## Getting started with Deep Learning:
    
- Pytorch Blitz: https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html
- Pytorch Documentation: https://pytorch.org/docs/stable/index.html

## Going further

**Question 1: The current model is trained without early stopping, split the records between the training set, validation set and test set and use early stopping to stop the training at the right moment.**

**Question 2: The model parameters are not optimized, by building a k-folds validation optimize the model hyperparameters**

**Question 3: As we have seen in the previous tutorial, spectrogram representation may be an efficient way to represent sleep. Build a model (CNN or RNN) which takes the spectrogram of the data as input. You can use the spectrogram from https://pytorch.org/audio/stable/transforms.html**

**Question 4: The current model only use a single EEG channel, what is the impact of using all the EEG channels ?**


**Additional challenge:** 

If you are done with sleep staging classification, you can try this other challenge on physiological data. The goal is to segment sleep apnea events based on PSG data.
https://challengedata.ens.fr/challenges/45